# Experimento 2

cGANS-MLP

In [10]:
# =========================================================
# main_experimento2.py
# Experimento 2 — cGAN MLP
# 1) Treina CNN com dados reais (baseline)
# 2) Treina cGAN MLP com dados reais
# 3) Gera dados sintéticos por proporção
# 4) Mistura dados reais + sintéticos
# 5) Treina CNN do zero
# 6) Avalia CNN apenas com dados reais
# 7) Calcula FID
# =========================================================

import torch
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torchvision import datasets, transforms
import numpy as np

# Modelos
from models import (
    CNNClassifier,
    ConditionalGenerator,
    ConditionalDiscriminator
)

# Rotinas
from training import (
    train_classifier,
    train_cgan,
    plot_confusion,
    compute_statistics,
    calculate_fid
)

# =========================================================
# CONFIGURAÇÃO
# =========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

# =========================================================
# DATASET MNIST
# =========================================================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(
    "./data", train=True, download=True, transform=transform
)

test_dataset = datasets.MNIST(
    "./data", train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# =========================================================
# BASELINE — CNN COM DADOS REAIS
# =========================================================
print("\n=== CNN BASELINE (DADOS REAIS) ===")

cnn_real = CNNClassifier().to(device)
train_classifier(cnn_real, train_loader, device)
plot_confusion(cnn_real, test_loader, device,
               "CNN treinada apenas com dados reais")

# =========================================================
# TREINAMENTO DA cGAN MLP
# =========================================================
print("\n=== TREINAMENTO DA cGAN MLP ===")

G = ConditionalGenerator().to(device)
D = ConditionalDiscriminator().to(device)

train_cgan(G, D, train_loader, device, epochs=50)

# =========================================================
# FUNÇÃO PARA GERAR DADOS SINTÉTICOS POR PROPORÇÃO
# =========================================================
def generate_fake_dataset_by_ratio(G, ratio, device):
    """
    Gera dados sintéticos balanceados por classe
    """
    n_real = len(train_dataset)
    n_fake = int(n_real * ratio)
    n_per_class = n_fake // 10

    images, labels = [], []

    G.eval()
    with torch.no_grad():
        for cls in range(10):
            z = torch.randn(n_per_class, 100, device=device)
            y = torch.full((n_per_class,), cls, device=device, dtype=torch.long)
            x_fake = G(z, y).cpu()
            images.append(x_fake)
            labels.append(y.cpu())

    images = torch.cat(images)
    labels = torch.cat(labels)

    return TensorDataset(images, labels)

# =========================================================
# FID — REAL x FAKE
# =========================================================
print("\n=== FID (REAL x FAKE) ===")

feature_extractor = cnn_real.features

mu_real, sigma_real = compute_statistics(
    train_loader, feature_extractor, device
)

# =========================================================
# LOOP DAS PROPORÇÕES
# =========================================================
ratios = [0.1, 0.3, 0.5]

for ratio in ratios:

    print(f"\n=== PROPORÇÃO DE DADOS SINTÉTICOS: {int(ratio*100)}% ===")

    # Gera dados fake
    fake_dataset = generate_fake_dataset_by_ratio(G, ratio, device)
    fake_loader = DataLoader(fake_dataset, batch_size=128, shuffle=False)

    # Calcula FID
    mu_fake, sigma_fake = compute_statistics(
        fake_loader, feature_extractor, device
    )

    fid_value = calculate_fid(
        mu_real, sigma_real, mu_fake, sigma_fake
    )

    print(f"FID (Real vs Fake {int(ratio*100)}%): {fid_value:.2f}")

    # Mistura dados reais + sintéticos
    mixed_dataset = ConcatDataset([train_dataset, fake_dataset])
    mixed_loader = DataLoader(
        mixed_dataset, batch_size=128, shuffle=True
    )

    # Treina CNN do zero
    cnn_mixed = CNNClassifier().to(device)
    train_classifier(cnn_mixed, mixed_loader, device)

    # Avaliação apenas com dados reais
    plot_confusion(
        cnn_mixed,
        test_loader,
        device,
        f"CNN com {int(ratio*100)}% dados sintéticos"
    )



=== CNN BASELINE (DADOS REAIS) ===


KeyboardInterrupt: 

In [9]:
# =========================================================
# models.py
# Contém apenas definições de ARQUITETURA CNN e cGANs
# Para cGANS utiliza uma MLP
# (nenhum treinamento aqui)
# =========================================================

# Importa o PyTorch base
import torch

# Importa o módulo de redes neurais
import torch.nn as nn

# =========================================================
# CNN CLASSIFICADORA PARA MNIST
# =========================================================
class CNNClassifier(nn.Module):
    """
    CNN simples usada para:
    - Classificação com dados reais
    - Classificação com dados fake gerados pela cGAN
    """

    def __init__(self):
        # Inicializa a classe base nn.Module
        super().__init__()

        # -------------------------
        # Extrator de características
        # -------------------------
        self.features = nn.Sequential(

            # Primeira convolução:
            # Entrada: 1 canal (imagem MNIST)
            # Saída: 32 mapas de características
            # kernel 3x3
            # Paramentros: numero de filtros=32, tamanho do kernel = 3
            nn.Conv2d(1, 32, kernel_size=3),

            # Função de ativação ReLU
            nn.ReLU(),

            # Reduz resolução espacial pela metade
            nn.MaxPool2d(2),
            #saida mapa 13x33

            # Segunda convolução:
            # Entrada: 32 mapas
            # Saída: 64 mapas
            nn.Conv2d(32, 64, kernel_size=3),

            # Ativação
            nn.ReLU(),

            # Novo downsampling
            nn.MaxPool2d(2)
        )
        # saida mapa 5x5

        # -------------------------
        # Classificador totalmente conectado
        # -------------------------
        self.classifier = nn.Sequential(

            # Achata o tensor 4D → 2D
            nn.Flatten(),

            # Camada totalmente conectada
            nn.Linear(64 * 5 * 5, 128),

            # Ativação
            nn.ReLU(),

            # Camada de saída (10 classes)
            nn.Linear(128, 10)
        )

    def forward(self, x):
        """
        Define o fluxo forward da CNN
        """

        # Extrai características
        x = self.features(x)

        # Realiza tarefa de Classificação
        x = self.classifier(x)

        return x


# =========================================================
# GERADOR CONDICIONAL (cGAN)
# =========================================================
class ConditionalGenerator(nn.Module):
    """
    Gerador da cGAN:
    Entrada:
      - vetor de ruído z
      - rótulo y
    Saída:
      - imagem 28x28 condicionada ao rótulo
    """

    def __init__(self, z_dim=100, n_classes=10):
        super().__init__()

        # Embedding transforma rótulos em vetores
        self.label_emb = nn.Embedding(
            num_embeddings=n_classes,
            embedding_dim=n_classes
        )

        # Rede totalmente conectada (MLP)
        self.net = nn.Sequential(

            # Entrada: ruído + rótulo
            nn.Linear(z_dim + n_classes, 256),

            nn.ReLU(),

            nn.Linear(256, 512),

            nn.ReLU(),

            # Saída: 28*28 pixels
            nn.Linear(512, 784),

            # Tanh → compatível com normalização [-1, 1]
            nn.Tanh()
        )

    def forward(self, z, y):
        """
        Forward do gerador
        """

        # Converte rótulo inteiro em vetor embedding
        y_emb = self.label_emb(y)

        # Concatena ruído e rótulo
        x = torch.cat([z, y_emb], dim=1)

        # Gera imagem e reorganiza para formato 2D
        img = self.net(x).view(-1, 1, 28, 28)

        return img


# =========================================================
# DISCRIMINADOR CONDICIONAL (cGAN)
# =========================================================
class ConditionalDiscriminator(nn.Module):
    """
    Discriminador da cGAN:
    Entrada:
      - imagem
      - rótulo
    Saída:
      - probabilidade de ser real
    """

    def __init__(self, n_classes=10):
        super().__init__()

        # Embedding do rótulo no espaço da imagem
        self.label_emb = nn.Embedding(
            num_embeddings=n_classes,
            embedding_dim=784
        )

        # Rede discriminadora - MLP
        self.net = nn.Sequential(

            nn.Linear(784 * 2, 512), #tamanho da imagem x (dados + classe)
                                      #vetor de classe codificado como 784 valores
            nn.LeakyReLU(0.2),

            nn.Linear(512, 256),

            nn.LeakyReLU(0.2),

            nn.Linear(256, 1),

            # Saída probabilística
            nn.Sigmoid()
        )

    def forward(self, x, y):
        """
        Forward do discriminador
        """

        # Achata imagem
        x = x.view(x.size(0), -1)

        # Embedding do rótulo
        y_emb = self.label_emb(y)

        # Concatena imagem + rótulo
        d_input = torch.cat([x, y_emb], dim=1)

        # Classificação real/fake
        out = self.net(d_input)

        return out


In [8]:
# =========================================================
# training.py
# Contém rotinas de treinamento e métricas
# =========================================================

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scipy import linalg
import matplotlib.pyplot as plt


# =========================================================
# TREINAMENTO DA CNN
# =========================================================
def train_classifier(model, loader, device, epochs=5):
    """
    Treina uma CNN supervisionada
    """

    # Coloca o modelo em modo treino
    model.train()

    # Otimizador Adam
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Função de perda multiclasse
    criterion = nn.CrossEntropyLoss()

    # Loop de épocas
    for epoch in range(epochs):

        # Loop sobre batches
        for x, y in loader:

            # Move dados para GPU/CPU
            x, y = x.to(device), y.to(device)

            # Zera gradientes acumulados
            optimizer.zero_grad()

            # Forward pass
            logits = model(x)

            # Calcula perda
            loss = criterion(logits, y)

            # Backpropagation
            loss.backward()

            # Atualiza pesos
            optimizer.step()

        print(f"[CNN] Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f}")


# =========================================================
# MATRIZ DE CONFUSÃO
# =========================================================
def plot_confusion(model, loader, device, title):
    """
    Calcula e plota a matriz de confusão
    """

    # Modo avaliação
    model.eval()

    y_true, y_pred = [], []

    # Sem gradientes
    with torch.no_grad():
        for x, y in loader:

            x = x.to(device)

            # Predição
            preds = model(x).argmax(dim=1).cpu()

            y_true.extend(y.numpy())
            y_pred.extend(preds.numpy())

    # Matriz de confusão
    cm = confusion_matrix(y_true, y_pred)

    # Visualização
    ConfusionMatrixDisplay(cm).plot()
    plt.title(title)
    plt.show()


# =========================================================
# TREINAMENTO DA cGAN
# =========================================================
def train_cgan(G, D, loader, device, epochs=20, z_dim=100):
    """
    Treina uma cGAN (gerador + discriminador)
    """

    # Otimizadores
    opt_g = optim.Adam(G.parameters(), lr=2e-4)
    opt_d = optim.Adam(D.parameters(), lr=2e-4)

    # Perda adversarial
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        for x, y in loader:

            x, y = x.to(device), y.to(device)
            batch = x.size(0)

            # ======================
            # Treina Discriminador
            # ======================
            z = torch.randn(batch, z_dim, device=device)

            fake = G(z, y)

            loss_d = (
                criterion(D(x, y), torch.ones(batch, 1, device=device)) +
                criterion(D(fake.detach(), y),
                          torch.zeros(batch, 1, device=device))
            )

            opt_d.zero_grad()
            loss_d.backward()
            opt_d.step()

            # ======================
            # Treina Gerador
            # ======================
            z = torch.randn(batch, z_dim, device=device)

            fake = G(z, y)

            loss_g = criterion(
                D(fake, y),
                torch.ones(batch, 1, device=device)
            )

            opt_g.zero_grad()
            loss_g.backward()
            opt_g.step()

        print(f"[cGAN] Epoch {epoch+1}/{epochs} | "
              f"D: {loss_d.item():.4f} | G: {loss_g.item():.4f}")


# =========================================================
# FID
# =========================================================
def compute_statistics(loader, extractor, device):
    """
    Calcula média e covariância das features
    """

    features = []

    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device)
            features.append(extractor(x).cpu().numpy())

    features = np.concatenate(features)

    return np.mean(features, axis=0), np.cov(features, rowvar=False)


def calculate_fid(mu1, sigma1, mu2, sigma2):
    """
    Calcula Fréchet Inception Distance
    """
    diff = mu1 - mu2
    covmean = linalg.sqrtm(sigma1 @ sigma2)



    # Correção numérica (parte imaginária pequena)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff @ diff + np.trace(sigma1 + sigma2 - 2 * covmean)

    return fid

cGANS-CONV

In [7]:
# =========================================================
# main_experimento2.py
# Experimento 2 — cGAN Convolucional (DC-cGAN)
# =========================================================

import sys
import os

sys.path.append(os.getcwd())

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset

# ======================
# Importação dos módulos
# ======================

# Executado a partir das celulas abaixo no colab.

# ======================
# Configurações
# ======================
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)

batch_size = 128
z_dim = 100
ratios = [0.1, 0.3, 0.5]

print(f"Dispositivo: {device}")

# ======================
# Transformações
# ======================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ======================
# Dataset MNIST
# ======================
train_dataset = datasets.MNIST(
    root="data", train=True, download=True, transform=transform
)

test_dataset = datasets.MNIST(
    root="data", train=False, download=True, transform=transform
)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

# =========================================================
# CNN BASELINE — DADOS REAIS
# =========================================================
print("\n=== CNN BASELINE (DADOS REAIS) ===")

cnn_real = CNNClassifier().to(device)
train_cnn(cnn_real, train_loader, device, epochs=10)

plot_confusion_matrix(
    cnn_real, test_loader, device,
    "CNN treinada apenas com dados reais"
)

# =========================================================
# TREINAMENTO DA cGAN CONVOLUCIONAL
# =========================================================
print("\n=== TREINAMENTO DA cGAN CONVOLUCIONAL ===")

G = CGANGenerator(z_dim=z_dim).to(device)
D = CGANDiscriminator().to(device)

train_cgan(
    G=G,
    D=D,
    dataloader=train_loader,
    device=device,
    epochs=30,
    z_dim=z_dim
)

# =========================================================
# FUNÇÃO: GERAR DADOS FAKE POR PROPORÇÃO
# =========================================================
def generate_fake_dataset_by_ratio(generator, ratio):
    n_real = len(train_dataset)
    n_fake = int(n_real * ratio)
    n_per_class = n_fake // 10

    images, labels = [], []

    generator.eval()
    with torch.no_grad():
        for cls in range(10):
            z = torch.randn(n_per_class, z_dim, device=device)
            y = torch.full((n_per_class,), cls, device=device, dtype=torch.long)
            fake_imgs = generator(z, y)

            images.append(fake_imgs.cpu())
            labels.append(y.cpu())

    images = torch.cat(images)
    labels = torch.cat(labels)

    return TensorDataset(images, labels)

# =========================================================
# FID — REAL × FAKE
# =========================================================
print("\n=== FID (REAL × FAKE) ===")

real_features = extract_features(
    model=cnn_real,
    dataloader=train_loader,
    device=device
)

mu_real, sigma_real = compute_statistics(real_features)

# =========================================================
# LOOP DAS PROPORÇÕES
# =========================================================
for ratio in ratios:

    print(f"\n=== PROPORÇÃO DE DADOS FAKE: {int(ratio*100)}% ===")

    # Dataset fake
    fake_dataset = generate_fake_dataset_by_ratio(G, ratio)
    fake_loader = DataLoader(
        fake_dataset, batch_size=batch_size, shuffle=False
    )

    # FID
    fake_features = extract_features(
        model=cnn_real,
        dataloader=fake_loader,
        device=device
    )

    mu_fake, sigma_fake = compute_statistics(fake_features)

    fid_value = calculate_fid(
        mu_real, sigma_real, mu_fake, sigma_fake
    )

    print(f"FID (Real vs Fake {int(ratio*100)}%): {fid_value:.2f}")

    # Dataset misto
    mixed_dataset = ConcatDataset([train_dataset, fake_dataset])
    mixed_loader = DataLoader(
        mixed_dataset, batch_size=batch_size, shuffle=True
    )

    # CNN treinada do zero
    cnn_mixed = CNNClassifier().to(device)
    train_cnn(cnn_mixed, mixed_loader, device, epochs=10)

    # Avaliação SOMENTE com dados reais
    plot_confusion_matrix(
        cnn_mixed,
        test_loader,
        device,
        f"CNN com {int(ratio*100)}% dados sintéticos (cGAN Conv)"
    )


Dispositivo: cpu

=== CNN BASELINE (DADOS REAIS) ===


KeyboardInterrupt: 

In [4]:
##EVALUATION:

#confusion_matriz.py
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataloader, device, title):
    """
    Calcula e plota a matriz de confusão de um classificador

    Parâmetros:
    - model: CNN treinada
    - dataloader: loader do conjunto de teste
    - device: cpu ou cuda
    - title: título do gráfico
    """

    model.eval()  # coloca o modelo em modo de avaliação

    all_preds = []   # lista para armazenar predições
    all_labels = []  # lista para armazenar rótulos reais

    # Desativa cálculo de gradiente (economia de memória)
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)

            # Classe predita = argmax dos logits
            preds = torch.argmax(outputs, dim=1)

            # Armazena resultados
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    # Concatena todos os batches
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Calcula a matriz de confusão
    cm = confusion_matrix(all_labels, all_preds)

    # Cria objeto de visualização
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)

    # Plota
    fig, ax = plt.subplots(figsize=(7, 7))
    disp.plot(ax=ax, cmap="Blues", colorbar=True)
    ax.set_title(title)

    plt.show()

#fid.py

import torch
import numpy as np
from scipy import linalg

def extract_features(model, dataloader, device):
    """
    Extrai features intermediárias de uma CNN para cálculo do FID

    Parâmetros:
    - model: CNN treinada (usada como extrator)
    - dataloader: DataLoader (real ou fake)
    - device: cpu ou cuda

    Retorno:
    - features: array numpy (N, D)
    """

    model.eval()               # modo avaliação
    features = []              # lista para armazenar features

    with torch.no_grad():      # desativa gradientes
        for images, _ in dataloader:
            images = images.to(device)

            # Forward apenas até a camada convolucional
            x = model.conv1(images)
            x = torch.relu(x)
            x = torch.max_pool2d(x, 2)

            x = model.conv2(x)
            x = torch.relu(x)
            x = torch.max_pool2d(x, 2)

            # Flatten das features
            x = x.view(x.size(0), -1)

            features.append(x.cpu().numpy())

    # Concatena todos os batches
    features = np.concatenate(features, axis=0)
    return features


def compute_statistics(features):
    """
    Calcula média e covariância das features

    Parâmetros:
    - features: array (N, D)

    Retorno:
    - mu: média (D,)
    - sigma: covariância (D, D)
    """

    mu = np.mean(features, axis=0)
    sigma = np.cov(features, rowvar=False)

    return mu, sigma


def calculate_fid(mu1, sigma1, mu2, sigma2):
    """
    Calcula Fréchet Inception Distance (FID)

    Fórmula:
    ||μ1 - μ2||² + Tr(Σ1 + Σ2 - 2(Σ1Σ2)¹ᐟ²)
    """

    diff = mu1 - mu2

    # Produto das covariâncias
    covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)

    # Correção numérica (parte imaginária pequena)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff @ diff + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

In [5]:
#MODELS:

#cgan_discriminator.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class CGANDiscriminator(nn.Module):
    """
    Discriminador convolucional condicional
    Entrada: imagem + mapa de classe
    Saída: probabilidade real/fake
    """
    def __init__(self, n_classes=10):
        super().__init__()

        # Embedding do rótulo como mapa espacial
        self.label_emb = nn.Embedding(n_classes, 28 * 28)

        self.conv = nn.Sequential(
            nn.Conv2d(2, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2)
        )

        self.fc = nn.Linear(128 * 7 * 7, 1)

    def forward(self, x, labels):
        # x: (B, 1, 28, 28)

        y = self.label_emb(labels)               # (B, 784)
        y = y.view(-1, 1, 28, 28)                # (B, 1, 28, 28)

        x = torch.cat([x, y], dim=1)             # (B, 2, 28, 28)

        x = self.conv(x)
        x = x.view(x.size(0), -1)

        return torch.sigmoid(self.fc(x))

#cgan_generator.py
import torch
import torch.nn as nn

class CGANGenerator(nn.Module):
    """
    Gerador convolucional condicional (DC-cGAN)
    Entrada: ruído z + rótulo y
    Saída: imagem 1x28x28
    """
    def __init__(self, z_dim=100, n_classes=10):
        super().__init__()

        # Embedding do rótulo (one-hot projetado)
        self.label_emb = nn.Embedding(n_classes, n_classes)

        # Camada inicial totalmente conectada
        self.fc = nn.Linear(z_dim + n_classes, 128 * 7 * 7)

        # Blocos convolucionais transpostos
        self.deconv = nn.Sequential(
            nn.BatchNorm2d(128),

            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 14x14
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 1, 4, stride=2, padding=1),    # 28x28
            nn.Tanh()
        )

    def forward(self, z, labels):
        # z: (B, 100)
        # labels: (B)

        y = self.label_emb(labels)              # (B, 10)
        x = torch.cat([z, y], dim=1)            # (B, 110)

        x = self.fc(x)                          # (B, 128*7*7)
        x = x.view(-1, 128, 7, 7)               # (B, 128, 7, 7)

        return self.deconv(x)                   # (B, 1, 28, 28)


#cnn.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    """
    CNN simples para classificação do MNIST
    Entrada: imagem 1x28x28
    Saída: logits para 10 classes
    """
    def __init__(self):
        super().__init__()

        # Bloco convolucional 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Camadas totalmente conectadas
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Entrada: (B, 1, 28, 28)

        x = F.relu(self.conv1(x))        # (B, 32, 28, 28)
        x = F.max_pool2d(x, 2)           # (B, 32, 14, 14)

        x = F.relu(self.conv2(x))        # (B, 64, 14, 14)
        x = F.max_pool2d(x, 2)           # (B, 64, 7, 7)

        x = x.view(x.size(0), -1)        # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)               # Logits

In [6]:
#TRAINING:

#train_cgan.py
import torch
import torch.nn as nn
import torch.optim as optim

def train_cgan(G, D, dataloader, device, epochs=30, z_dim=100):
    """
    Treinamento padrão de uma DC-cGAN
    """
    criterion = nn.BCELoss()
    opt_g = optim.Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
    opt_d = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.999))

    for epoch in range(epochs):
        for real_imgs, labels in dataloader:
            real_imgs, labels = real_imgs.to(device), labels.to(device)
            batch = real_imgs.size(0)

            # Rótulos reais e falsos
            valid = torch.ones(batch, 1, device=device)
            fake = torch.zeros(batch, 1, device=device)

            # ======================
            # Treina Discriminador
            # ======================
            z = torch.randn(batch, z_dim, device=device)
            gen_imgs = G(z, labels)

            d_real = D(real_imgs, labels)
            d_fake = D(gen_imgs.detach(), labels)

            loss_d = criterion(d_real, valid) + criterion(d_fake, fake)

            opt_d.zero_grad()
            loss_d.backward()
            opt_d.step()

            # ======================
            # Treina Gerador
            # ======================
            z = torch.randn(batch, z_dim, device=device)
            gen_imgs = G(z, labels)

            loss_g = criterion(D(gen_imgs, labels), valid)

            opt_g.zero_grad()
            loss_g.backward()
            opt_g.step()

        print(f"Epoch [{epoch+1}/{epochs}] | D: {loss_d.item():.4f} | G: {loss_g.item():.4f}")


#train_cnn.py
import torch
import torch.nn as nn
import torch.optim as optim

def train_cnn(model, dataloader, device, epochs=10):
    """
    Treinamento supervisionado da CNN
    """
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    model.train()

    for epoch in range(epochs):
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {loss.item():.4f}")